# Website Traffic Prediction With Colaboratory and Facebook Prophet

A few resources to read up on Prophet:


*   https://research.fb.com/prophet-forecasting-at-scale/
*   https://facebook.github.io/prophet/
*   http://pbpython.com/prophet-overview.html



## Install Needed Libraries In Colab

In [ ]:
# Libraries
import pandas as pd
import numpy as np
import re
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY

import matplotlib.pyplot as plt

import json
import forecastga
import forecastga.googleanalytics as ga


# Logging
import logging
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)

## Import Needed libraries

In [ ]:
from types import SimpleNamespace

with open('identity.json') as f:
    jf = json.load(f)
    identify_json = SimpleNamespace(**jf)



In [ ]:
# Libraries
import pandas as pd
import numpy as np
import re
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY

import matplotlib.pyplot as plt

import json
import forecastga
import forecastga.googleanalytics as ga


# Logging
import logging
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)



from types import SimpleNamespace

with open('identity.json') as f:
    jf = json.load(f)
    identify_json = SimpleNamespace(**jf)


#@title Google Analytics
ga_url = "https://analytics.google.com/analytics/web/?authuser=2#/report-home/a49839941w81675857p84563570" #@param {type:"string"}
ga_segment = "organic traffic" #@param ["all users", "organic traffic", "direct traffic", "referral traffic", "mobile traffic", "tablet traffic"] {type:"string"}
ga_metric = "sessions" #@param ["sessions", "pageviews", "unique pageviews", "transactions"] {type:"string"}


# @title Historical Data

# @markdown #### Date Range:
ga_start_date = "2018-01-01" #@param {type:"date"}
ga_end_date = "2019-12-31" #@param {type:"date",name:"GA Date"}

# @markdown ***
# @markdown <div align="center">OR</div>

# @markdown #### Prior Months:
prior_months = 0 #@param {type:"integer"}

# @title Prediction Data

future_months = 2 # @param {type:"slider", min:1, max:24, step:1}
#@markdown ---
#@markdown `max_available_volume` is the total possible daily volume for a niche/geo. This helps keep the algorithm honest by putting a max possible upper bound on prediction.
max_available_volume = 12222 # @param {type:"integer", hint:"this is a description"}
#@markdown ---
#@markdown `omit_values_over` is a way to clean your existing data to remove one-time spikes, caused by a rare event that is unlikely to happen again.  This keeps the algorithm from using this data in its future predictions.
omit_values_over = 2000000 # @param {type:"integer"}
#@markdown ---
save_output = False # @param {type:"boolean"}



try:
    profile = ga.authenticate(
      client_id=identify_json.client_id,
      client_secret=identify_json.client_secret,
      identity=identify_json.identity,
      ga_url=ga_url,
      interactive=True
  )
    print('Authenticated')
except Exception as e:
    print('An error occured', str(e))


class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

def p_date(_dt):
    return datetime.strftime(_dt, '%Y-%m-%d')

def get_months(start_date,end_date ):
    strt_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    return rrule(MONTHLY, dtstart=strt_dt, until=end_dt).count()


def print_profiles(accounts):

    out = []

    for a in accounts:
        out.append({"accounts":a.name, "webproperties":""})
    for w in a.webproperties:
        out.append({"accounts": "", "webproperties":w.name})

    display(pd.DataFrame(out,columns= ['accounts','webproperties']))


def get_ga_data(profile, data):

    try:

        if data.prior_months and int(data.prior_months) > 0:
            print('Pulling {} prior months data.'.format(data.prior_months))
            sessions = \
                profile.core.query.metrics(data.ga_metric).segment(data.ga_segment).daily(months=0
                    - int(data.prior_months)).report
        else:
            print('Pulling data from {} to {}.'.format(data.ga_start_date, data.ga_end_date))
            sessions = \
                profile.core.query.metrics(data.ga_metric).segment(data.ga_segment).daily(data.ga_start_date,
                    data.ga_end_date).report

    except Exception as e:
        print ('Error. Error retreiving data from Google Analytics.', str(e))
        return None

    df = sessions.as_dataframe()

    df['date'] = pd.to_datetime(df['date'])


    # Clean data.
    if data.omit_values_over and int(data.omit_values_over) > 0:
        df.loc[df[data.ga_metric] > data.omit_values_over, data.ga_metric] = np.nan

    df.loc[df[data.ga_metric] < 1, data.ga_metric] = np.nan

    df.dropna(inplace=True, axis=0)


    print('Rows: {rows} Min Date: {min_date} Max Date: {max_date}'.format(rows=len(df),
                                                                      min_date=p_date(df.date.min()),
                                                                      max_date=p_date(df.date.max())
                                                                      ))
    # Backfilling missing values
    df = df.set_index('date').asfreq('d', method='bfill')

    return df


data = Struct(**{
    'ga_segment': ga_segment,
    'ga_metric': ga_metric,
    'ga_start_date': ga_start_date,
    'ga_end_date': ga_end_date,
    'prior_months': prior_months,
    'omit_values_over': omit_values_over,
    })


datafile = get_ga_data(profile, data)
datafile.plot()


In [ ]:
model_list = ["ARIMA","Gluonts"]

am = forecastga.AutomatedModel(df = datafile , model_list=model_list, forecast_len=30 )

In [ ]:
am.forecast_insample()

## Let's Collect Some Data

### Account Settings:

In [ ]:
#@title Google Analytics
ga_url = "https://analytics.google.com/analytics/web/?authuser=2#/report-home/a49839941w81675857p84563570" #@param {type:"string"}
ga_segment = "organic traffic" #@param ["all users", "organic traffic", "direct traffic", "referral traffic", "mobile traffic", "tablet traffic"] {type:"string"}
ga_metric = "sessions" #@param ["sessions", "pageviews", "unique pageviews", "transactions"] {type:"string"}

### Timeframe Settings:

In [ ]:
# @title Historical Data

# @markdown #### Date Range:
ga_start_date = "2018-01-01" #@param {type:"date"}
ga_end_date = "2019-12-31" #@param {type:"date",name:"GA Date"}

# @markdown ***
# @markdown <div align="center">OR</div>

# @markdown #### Prior Months:
prior_months = 0 #@param {type:"integer"}


### Prediction Settings:

In [ ]:
# @title Prediction Data

future_months = 2 # @param {type:"slider", min:1, max:24, step:1}
#@markdown ---
#@markdown `max_available_volume` is the total possible daily volume for a niche/geo. This helps keep the algorithm honest by putting a max possible upper bound on prediction.
max_available_volume = 12222 # @param {type:"integer", hint:"this is a description"}
#@markdown ---
#@markdown `omit_values_over` is a way to clean your existing data to remove one-time spikes, caused by a rare event that is unlikely to happen again.  This keeps the algorithm from using this data in its future predictions. 
omit_values_over = 2000000 # @param {type:"integer"}
#@markdown ---
save_output = False # @param {type:"boolean"}

## Authenticate

In [ ]:
try:
    profile = ga.authenticate(
      client_id=identify_json.client_id, 
      client_secret=identify_json.client_secret, 
      identity=identify_json.identity,
      ga_url=ga_url,
      interactive=True
  )
    print('Authenticated')
except Exception as e:
    print('An error occured', str(e))

## Get GA Time Series Data

In [ ]:
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

def p_date(_dt):
    return datetime.strftime(_dt, '%Y-%m-%d')

def get_months(start_date,end_date ):
    strt_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    return rrule(MONTHLY, dtstart=strt_dt, until=end_dt).count()


def print_profiles(accounts):

    out = []

    for a in accounts: 
        out.append({"accounts":a.name, "webproperties":""})
    for w in a.webproperties:
        out.append({"accounts": "", "webproperties":w.name})

    display(pd.DataFrame(out,columns= ['accounts','webproperties']))


def get_ga_data(profile, data):

    try:

        if data.prior_months and int(data.prior_months) > 0:
            print('Pulling {} prior months data.'.format(data.prior_months))
            sessions = \
                profile.core.query.metrics(data.ga_metric).segment(data.ga_segment).daily(months=0
                    - int(data.prior_months)).report
        else:
            print('Pulling data from {} to {}.'.format(data.ga_start_date, data.ga_end_date))
            sessions = \
                profile.core.query.metrics(data.ga_metric).segment(data.ga_segment).daily(data.ga_start_date,
                    data.ga_end_date).report

    except Exception as e:
        print ('Error. Error retreiving data from Google Analytics.', str(e))
        return None

    df = sessions.as_dataframe()

    df['date'] = pd.to_datetime(df['date'])


    # Clean data.
    if data.omit_values_over and int(data.omit_values_over) > 0:
        df.loc[df[data.ga_metric] > data.omit_values_over, data.ga_metric] = np.nan

    df.loc[df[data.ga_metric] < 1, data.ga_metric] = np.nan

    df.dropna(inplace=True, axis=0)


    print('Rows: {rows} Min Date: {min_date} Max Date: {max_date}'.format(rows=len(df),
                                                                      min_date=p_date(df.date.min()),
                                                                      max_date=p_date(df.date.max())
                                                                      ))
    # Backfilling missing values
    df = df.set_index('date').asfreq('d', method='bfill') 

    return df


data = Struct(**{
    'ga_segment': ga_segment,
    'ga_metric': ga_metric,
    'ga_start_date': ga_start_date,
    'ga_end_date': ga_end_date,
    'prior_months': prior_months,
    'omit_values_over': omit_values_over,
    })


datafile = get_ga_data(profile, data)

def plot_colab(df, title=None, dark_mode=False):

    from IPython.display import Markdown as md
    plt.style.use('default')

    def show_md(txt):
        display(md(txt))

    if dark_mode:
        # Good all around color library
        plt.style.use('seaborn-colorblind')
        plt.rcParams.update({
            "lines.color": "#565555",
            "legend.edgecolor": "#818080",
            "legend.borderpad": 0.6,
            "text.color": "white",
            "axes.facecolor": "#383838",
            "axes.edgecolor": "#565555",
            "axes.grid": True,
            "axes.labelcolor": "white",
            "grid.color": "#565555",
            "xtick.color": "white",
            "ytick.color": "white",
            "figure.facecolor": "#383838",
            "savefig.facecolor": "white",
            "savefig.edgecolor": "white",
            "font.sans-serif": "Liberation Sans",
            "lines.linewidth": 2,
            "figure.figsize": [15,10],
            "font.size": 16
                    })
        
    else:
        plt.style.use('seaborn-colorblind')
        plt.rcParams.update({
            "legend.borderpad": 0.6,
            "axes.grid": True,
            "font.sans-serif": "Liberation Sans",
            "lines.linewidth": 2,
            "figure.figsize": [15,10],
            "font.size": 16
                    })

    if title:
        show_md('## {}'.format(title))

    df.plot()
    plt.show()



#plot_colab(datafile, title="Trending Data", dark_mode=True)

datafile.plot()



In [ ]:
sessions = profile.core.query.metrics('Sessions').daily(months=0
                    - int(4)).report

## Forecasting


In [ ]:

model_list = ["Prophet","NBEATS", "TATS", "TBATS1", "TBATP1", "TBATS2", "ARIMA","Gluonts"]

am = AutomatedModel(df = datafile , model_list=model_list, forecast_len=180 )

In [ ]:
forecast_in, performance = am.forecast_insample()

#### Automated Models

1. ```ARIMA``` - Automated ARIMA Modelling
1. ```Prophet``` - Modeling Multiple Seasonality With Linear or Non-linear Growth
1. ```HWAAS``` - Exponential Smoothing With Additive Trend and Additive Seasonality
1. ```HWAMS``` - Exponential Smoothing with Additive Trend and Multiplicative Seasonality
1. ```NBEATS``` -  Neural basis expansion analysis (now fixed at 20 Epochs)
1. ```Gluonts``` - RNN-based Model (now fixed at 20 Epochs)
1. ```TATS``` - Seasonal and Trend no Box Cox
1. ```TBAT``` - Trend and Box Cox
1. ```TBATS1``` - Trend, Seasonal (one), and Box Cox
1. ```TBATP1``` - TBATS1 but Seasonal Inference is Hardcoded by Periodicity
1. ```TBATS2``` - TBATS1 With Two Seasonal Periods